### Part 1 : Data Scraping and preparation

##### Below code performs data scraping data from espncricinfo.com. Data has been scraped across 8 pages(starting 2005 Jan) and for these  cricket teams : Australia, South Africa, West Indies, India, Sri Lanka, Pakistan, New Zealand , Bangladesh since these are the teams that qualified for the Quarter Finals of the Cricket World cup held in 2015

In [2]:
import csv
import lxml.html
import pandas as pd
import numpy as np

##### Gather data all One Day International(aka ODI) matches played from 01 Jan 2005 till date,include only games won or lost

In [7]:
url1='http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;'
url2 ='page='
url3 ='result=1;result=2;size=200;spanval2=span;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=batting;view=results'
url5 = ['http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;result=1;result=2;size=200;spanval2=span;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=batting;view=results']
for i in range(2,30):
    url4 = url1 + url2 + str(i) + ';' + url3
    url5.append(url4)
    
#http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;result=2;size=200;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=batting

##### Below piece of code crawls over the html page to derive various fields and writes them to a csv file

In [4]:
out = csv.writer(open('ODI_Batting_DataSet.csv','wb',))
out.writerow(('Mat','Inns','NO','Runs','HS','Ave','BF','SR','Centuries', 'Fifties', 'Ducks'))

for page in url5:
    team = []
    mat_res = []
    mgn = []
    br = []
    toss=[]
    bat=[]
    oppos = []
    bat_grd = []
    startdate = []
    
    content = lxml.html.parse(page)
    
    tm = content.xpath('//tr[@class="data1"]/td[1]/a')
    mat_result = content.xpath('//tr[@class="data1"]/td[2]')
    margin = content.xpath('//tr[@class="data1"]/td[3]')
    b_rate = content.xpath('//tr[@class="data1"]/td[4]')
    ts = content.xpath('//tr[@class="data1"]/td[5]')
    bt = content.xpath('//tr[@class="data1"]/td[6]')
    opp = content.xpath('//tr[@class="data1"]/td[8]/a')
    grd=content.xpath('//tr[@class="data1"]/td[9]/a')
    sdate=content.xpath('//tr[@class="data1"]/td[10]/b')
    
    tms = [t.text for t in tm]
    mr = [m.text for m in mat_result]
    mg = [mn.text for mn in margin]
    brt = [b.text for b in b_rate]
    toss_res=[to.text for to in ts]
    batting=[batt.text for batt in bt]
    opposit = [o.text for o in opp]
    gd = [g.text for g in grd]
    sd = [s.text for s in sdate]
    
    team.extend(tms)
    mat_res.extend(mr)
    mgn.extend(mg)
    br.extend(brt)
    toss.extend(toss_res)
    bat.extend(batting)
    oppos.extend(opposit)
    bat_grd.extend(gd)
    startdate.extend(sd)
    
    # This part of code may not be needed
    zipped = zip(team,mat_res,mgn,br,toss,bat,oppos,bat_grd,startdate)
    for row in zipped:
        out.writerow(row)
        zipped = None    

##### Here is how the extracted data looks like

In [3]:
df=pd.read_csv(r"ODI_DataSet.csv")
df.head()

,Team,Result,Margin,BR,Toss,Bat,Opposition,Ground,StartDate
0,Australia,won,116 runs,NaN,won,1st,West Indies,Melbourne,14 Jan 2005
1,West Indies,lost,116 runs,NaN,lost,2nd,Australia,Melbourne,14 Jan 2005
2,Australia,won,4 wickets,12,lost,2nd,Pakistan,Hobart,16 Jan 2005
3,Pakistan,lost,4 wickets,12,won,1st,Australia,Hobart,16 Jan 2005
4,Pakistan,won,6 wickets,18,won,2nd,West Indies,Brisbane,19 Jan 2005


##### Below code calculates new columns based on extracted data. These columns include Win/Loss flag,cumulative win count for each team, and % Win for each team, starting from 2005 Jan 1

In [4]:
df['Game_Win_Flag']=df['Result'].map(lambda x: 1 if x == 'won' else 0)
#Team 1
df_aus=df.query('Team == "Australia"')
df_aus_cnt=df.query('Team == "Australia"').count()
df_aus['win_loss_flag']=df_aus['Result'].map(lambda x: 1 if x == 'won' else 0)
df_aus['cum_sum'] = df_aus.win_loss_flag.cumsum()
df_aus['cum_sum_shifted']=df_aus['cum_sum'].shift(1)
df_aus['cum_wins']=np.nan_to_num(df_aus['cum_sum_shifted'])
df_aus['counter']=np.arange(df_aus_cnt['Team'])
df_aus['Pct_win']=df_aus.cum_wins/df_aus.counter
aus=df_aus[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
aus.to_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv")

#Team 2
df_sa=df.query('Team == "South Africa"')
df_sa_cnt=df.query('Team == "South Africa"').count()
df_sa['win_loss_flag']=df_sa['Result'].map(lambda x: 1 if x == 'won' else 0)
df_sa['cum_sum'] = df_sa.win_loss_flag.cumsum()
df_sa['cum_sum_shifted']=df_sa['cum_sum'].shift(1)
df_sa['cum_wins']=np.nan_to_num(df_sa['cum_sum_shifted'])
df_sa['counter']=np.arange(df_sa_cnt['Team'])
df_sa['Pct_win']=df_sa.cum_wins/df_sa.counter
sa=df_sa[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
sa.to_csv(f, header = False)
f.close()

#Team 3
df_sl=df.query('Team == "Sri Lanka"')
df_sl_cnt=df.query('Team == "Sri Lanka"').count()
df_sl['win_loss_flag']=df_sl['Result'].map(lambda x: 1 if x == 'won' else 0)
df_sl['cum_sum'] = df_sl.win_loss_flag.cumsum()
df_sl['cum_sum_shifted']=df_sl['cum_sum'].shift(1)
df_sl['cum_wins']=np.nan_to_num(df_sl['cum_sum_shifted'])
df_sl['counter']=np.arange(df_sl_cnt['Team'])
df_sl['Pct_win']=df_sl.cum_wins/df_sl.counter
sl=df_sl[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
sl.to_csv(f, header = False)
f.close()

#Team 4
df_pak=df.query('Team == "Pakistan"')
df_pak_cnt=df.query('Team == "Pakistan"').count()
df_pak['win_loss_flag']=df_pak['Result'].map(lambda x: 1 if x == 'won' else 0)
df_pak['cum_sum'] = df_pak.win_loss_flag.cumsum()
df_pak['cum_sum_shifted']=df_pak['cum_sum'].shift(1)
df_pak['cum_wins']=np.nan_to_num(df_pak['cum_sum_shifted'])
df_pak['counter']=np.arange(df_pak_cnt['Team'])
df_pak['Pct_win']=df_pak.cum_wins/df_pak.counter
pak=df_pak[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
pak.to_csv(f, header = False)
f.close()

#Team 5
df_bl=df.query('Team == "Bangladesh"')
df_bl_cnt=df.query('Team == "Bangladesh"').count()
df_bl['win_loss_flag']=df_bl['Result'].map(lambda x: 1 if x == 'won' else 0)
df_bl['cum_sum'] = df_bl.win_loss_flag.cumsum()
df_bl['cum_sum_shifted']=df_bl['cum_sum'].shift(1)
df_bl['cum_wins']=np.nan_to_num(df_bl['cum_sum_shifted'])
df_bl['counter']=np.arange(df_bl_cnt['Team'])
df_bl['Pct_win']=df_bl.cum_wins/df_bl.counter
bangla=df_bl[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
bangla.to_csv(f, header = False)
f.close()

#Team 6
df_wi=df.query('Team == "West Indies"')
df_wi_cnt=df.query('Team == "West Indies"').count()
df_wi['win_loss_flag']=df_wi['Result'].map(lambda x: 1 if x == 'won' else 0)
df_wi['cum_sum'] = df_wi.win_loss_flag.cumsum()
df_wi['cum_sum_shifted']=df_wi['cum_sum'].shift(1)
df_wi['cum_wins']=np.nan_to_num(df_wi['cum_sum_shifted'])
df_wi['counter']=np.arange(df_wi_cnt['Team'])
df_wi['Pct_win']=df_wi.cum_wins/df_wi.counter
wi=df_wi[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
wi.to_csv(f, header = False)
f.close()

#Team 7
df_nz=df.query('Team == "New Zealand"')
df_nz_cnt=df.query('Team == "New Zealand"').count()
df_nz['win_loss_flag']=df_nz['Result'].map(lambda x: 1 if x == 'won' else 0)
df_nz['cum_sum'] = df_nz.win_loss_flag.cumsum()
df_nz['cum_sum_shifted']=df_nz['cum_sum'].shift(1)
df_nz['cum_wins']=np.nan_to_num(df_nz['cum_sum_shifted'])
df_nz['counter']=np.arange(df_nz_cnt['Team'])
df_nz['Pct_win']=df_nz.cum_wins/df_nz.counter
nz=df_nz[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
nz.to_csv(f, header = False)
f.close()

#Team 8
df_in=df.query('Team == "India"')
df_in_cnt=df.query('Team == "India"').count()
df_in['win_loss_flag']=df_in['Result'].map(lambda x: 1 if x == 'won' else 0)
df_in['cum_sum'] = df_in.win_loss_flag.cumsum()
df_in['cum_sum_shifted']=df_in['cum_sum'].shift(1)
df_in['cum_wins']=np.nan_to_num(df_in['cum_sum_shifted'])
df_in['counter']=np.arange(df_in_cnt['Team'])
df_in['Pct_win']=df_in.cum_wins/df_in.counter
india=df_in[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','Pct_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv", 'a') # Open file as append mode
india.to_csv(f, header = False)
f.close()

/Users/hadoop/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hadoop/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hadoop/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.py

##### This was executed for each combination of teams(for 8 teams), to determine the % head-to-head win, starting from 2005

In [5]:
df=pd.read_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed.csv")
df['Game_Win_Flag']=df['Result'].map(lambda x: 1 if x == 'won' else 0)
#Team 1
df_res=df.query('Team == "West Indies" & Opposition=="New Zealand"')
df_res_cnt=df.query('Team == "West Indies" & Opposition=="New Zealand"').count()
df_res['win_loss_flag']=df_res['Result'].map(lambda x: 1 if x == 'won' else 0)
df_res['cum_sum'] = df_res.win_loss_flag.cumsum()
df_res['cum_sum_shifted']=df_res['cum_sum'].shift(1)
df_res['cum_h2h_wins']=np.nan_to_num(df_res['cum_sum_shifted'])
df_res['counter_h2h']=np.arange(df_res_cnt['Team'])
df_res['Pct_head_to_head_win']=df_res.cum_h2h_wins/df_res.counter_h2h
res=df_res[['Team','Result','Margin','BR','Toss','Bat','Opposition','Ground','StartDate','cum_wins','counter','cum_h2h_wins','counter_h2h','Pct_win','Pct_head_to_head_win']]
f = open(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed_Data.csv", 'a') # Open file as append mode
res.to_csv(f, header = True)
f.close()

/Users/hadoop/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hadoop/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/hadoop/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.py

In [6]:
df=pd.read_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\ODI_DataSet_Processed_Data.csv")
df['Toss_Win_Flag']= df['Toss'].map(lambda x: 1 if x == 'won' else 0)
df['Game_Win_Flag']= df['Result'].map(lambda x: 1 if x == 'won' else 0)
df['Bat_First_Flag']= df['Bat'].map(lambda x: 1 if x == '1st' else 0)
df.to_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\Analysis_DataSet.csv")

##### Dummy variables are created for each team and the opposition, in order to prepare data for regression.  Since there are 8 teams, 7 dummy variables are captured for 'Team' and 'Opposition' columns

In [7]:
df=pd.read_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\Analysis_DataSet.csv")
df['Team_Aus']=df['Team'].map(lambda x: 1 if x == 'Australia' else 0)
df['Team_Ind']=df['Team'].map(lambda x: 1 if x == 'India' else 0)
df['Team_SA']=df['Team'].map(lambda x: 1 if x == 'South Africa' else 0)
df['Team_NZ']=df['Team'].map(lambda x: 1 if x == 'New Zealand' else 0)
df['Team_SL']=df['Team'].map(lambda x: 1 if x == 'Sri Lanka' else 0)
df['Team_Pak']=df['Team'].map(lambda x: 1 if x == 'Pakistan' else 0)
df['Team_WI']=df['Team'].map(lambda x: 1 if x == 'West Indies' else 0)

df['Opposition_Aus']=df['Opposition'].map(lambda x: 1 if x == 'Australia' else 0)
df['Opposition_Ind']=df['Opposition'].map(lambda x: 1 if x == 'India' else 0)
df['Opposition_SA']=df['Opposition'].map(lambda x: 1 if x == 'South Africa' else 0)
df['Opposition_NZ']=df['Opposition'].map(lambda x: 1 if x == 'New Zealand' else 0)
df['Opposition_SL']=df['Opposition'].map(lambda x: 1 if x == 'Sri Lanka' else 0)
df['Opposition_Pak']=df['Opposition'].map(lambda x: 1 if x == 'Pakistan' else 0)
df['Opposition_WI']=df['Opposition'].map(lambda x: 1 if x == 'West Indies' else 0)
df.to_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\Final_Data\Analysis_DataSet.csv")

In [8]:
pd.read_csv(r"C:\data\Project\Datasets\Data_Regression\ODI_2005_Onwards\Final_Data\Analysis_DataSet.csv").head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,Team,Result,Margin,BR,Toss,Bat,Opposition,...,Team_SL,Team_Pak,Team_WI,Opposition_Aus,Opposition_Ind,Opposition_SA,Opposition_NZ,Opposition_SL,Opposition_Pak,Opposition_WI
0,0,852,852,West Indies,lost,81 runs,NaN,lost,2nd,New Zealand,...,0,0,1,0,0,0,1,0,0,0
1,1,853,853,West Indies,lost,3 wickets,48,lost,1st,New Zealand,...,0,0,1,0,0,0,1,0,0,0
2,2,854,854,West Indies,lost,21 runs,NaN,lost,2nd,New Zealand,...,0,0,1,0,0,0,1,0,0,0
3,3,855,855,West Indies,lost,91 runs,NaN,won,2nd,New Zealand,...,0,0,1,0,0,0,1,0,0,0
4,4,856,856,West Indies,won,3 wickets,2,lost,2nd,New Zealand,...,0,0,1,0,0,0,1,0,0,0
5,5,883,883,West Indies,lost,7 wickets,64,lost,1st,New Zealand,...,0,0,1,0,0,0,1,0,0,0
6,6,902,902,West Indies,won,5 wickets,1,won,2nd,New Zealand,...,0,0,1,0,0,0,1,0,0,0
7,7,903,903,West Indies,lost,7 wickets,177,lost,1st,New Zealand,...,0,0,1,0,0,0,1,0,0,0
8,8,904,904,West Indies,lost,9 runs,NaN,lost,1st,New Zealand,...,0,0,1,0,0,0,1,0,0,0
9,9,951,951,West Indies,won,9 wickets,52,won,2nd,New Zealand,...,0,0,1,0,0,0,1,0,0,0
